# Juliaで固有値問題を精度保証付き数値計算したい


## 固有値問題とは

固有値問題は与えられた行列の固有値および固有ベクトルを求めること。つまり、
$
\newcommand{\C}{\mathbb{C}}
\newcommand{\IC}{\mathbb{IC}}
\newcommand{\bX}{\bar{X}}
$

$$
Ax=\lambda x,\quad A\in \C^{n\times n}
$$

を満たす$\lambda\in \C$, $x\in \C^{n\times n}$ ($x\neq 0$) を求める問題を指す。

Juliaで固有値と固有ベクトルを求める際には、`LinearAlgebra`パッケージに含まれている関数`eigen`・`eigvals`・`eigvecs`で求めることができる。

|関数| 役割 |
|:-----------|:----------|
| eigen()    | 固有値・固有ベクトル両方  |
| eigvals()    | 固有値  |
| eigvecs()  | 固有ベクトル |

多分、Lapackの何かを呼び足してるはず。

In [43]:
using LinearAlgebra
val, vec  = eigen([1 2; 2 1])

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
2-element Array{Float64,1}:
 -1.0
  3.0
vectors:
2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [45]:
val#固有値

2-element Array{Float64,1}:
 -1.0
  3.0

In [3]:
vec#固有ベクトル

2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

In [4]:
eigvals([1 2; 2 1])#固有値

2-element Array{Float64,1}:
 -1.0
  3.0

In [5]:
eigvecs([1 2; 2 1])#固有ベクトル

2×2 Array{Float64,2}:
 -0.707107  0.707107
  0.707107  0.707107

## 密行列の固有値問題に対する精度保証

密行列とは、行列内のほとんどの要素が0でない行列のことを指す。ここでは、行列$A\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ の全ての固有値に対する精度保証アルゴリズムを考える。 

### 実装したいアルゴリズム
(1)
ある行列$A$の全ての近似固有ベクトル$\bX\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ を求める。

(2)
$A\cdot\bX$ の包含 $C\in\IC^{n\times n}$ を求める。

(3)
区間連立1次方程式 $\bX G=C$ の解集合の包含 $G\supset\bX^{-1}\cdot C$ を求める。

(4)
$G\in\IC^{n\times n}$ に対してゲルシュゴリンの定理を適用することを考える。

#### (1)行列$A$の全ての近似固有ベクトル$\bX\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ を求める。

行列$A\in\C^{n\times n}$ $(i=1,2,\cdots,n)$ が対角化可能であるとするとき、$\lambda_i$ に対する固有ベクトル$x^{(i)}\in\C^{n}$ $(i=1,2,\cdots,n)$ を並べた行列

$$
X :=[x^{(1)},x^{(2)},\cdots,x^{(n)}]\in\C^{n\times n}
$$

によって、$A$ は

$$
X^{-1}AX = D = \rm{diag}(\lambda_1,\lambda_2,\cdots,\lambda_n)
$$

と対角化される。

In [1]:
using LinearAlgebra

n =100;
A = randn(n,n);

λ, X = eigen(A);

In [2]:
λ #固有値

100-element Array{Complex{Float64},1}:
 -10.626975735953893 + 0.0im
  -8.743512787633641 - 1.979586000857552im
  -8.743512787633641 + 1.979586000857552im
  -8.740445960075004 + 0.0im
  -7.783469051758448 - 3.0262855307800436im
  -7.783469051758448 + 3.0262855307800436im
  -7.298423793683931 - 5.348790486047355im
  -7.298423793683931 + 5.348790486047355im
  -6.692017794617759 - 1.9496865160427062im
  -6.692017794617759 + 1.9496865160427062im
  -6.442644163177895 - 3.2444606706596173im
  -6.442644163177895 + 3.2444606706596173im
 -6.1942651507751645 - 4.631973629851082im
                     ⋮
   6.539332578855065 + 2.233032075725359im
   6.607228999015904 - 6.165445801895943im
   6.607228999015904 + 6.165445801895943im
    7.22169921106108 - 0.9377866741033362im
    7.22169921106108 + 0.9377866741033362im
   7.710962406320883 - 3.2613175020959546im
   7.710962406320883 + 3.2613175020959546im
  7.9204533755907915 + 0.0im
   8.389166711396669 + 0.0im
   9.654718397041066 - 2.5368682376154

In [3]:
X #固有ベクトル

100×100 Array{Complex{Float64},2}:
   0.0501083+0.0im    0.0698407-0.0889616im   …  -0.00810006+0.0im
  0.00532121+0.0im    0.0326682-0.0391103im       -0.0698515+0.0im
   0.0742684+0.0im   -0.0484232-0.0138022im         0.114154+0.0im
   -0.114341+0.0im   -0.0917039-0.04303im           0.102704+0.0im
  0.00291354+0.0im    0.0857744-0.0352752im        -0.043013+0.0im
   0.0265102+0.0im   -0.0405371-0.0074808im   …     0.137271+0.0im
  -0.0601424+0.0im   -0.0431608+0.135273im        -0.0497497+0.0im
  -0.0840584+0.0im    0.0649642+0.0714587im        -0.143419+0.0im
   0.0713203+0.0im   -0.0561176-0.0289629im      -0.00568682+0.0im
   0.0806892+0.0im   -0.0507801+0.0116821im        -0.138146+0.0im
   -0.162551+0.0im  -0.00321608-0.0420719im   …    -0.163856+0.0im
   0.0125974+0.0im    0.0558941-0.0561896im       -0.0776132+0.0im
    0.186565+0.0im    -0.101876+0.0126954im        0.0461539+0.0im
            ⋮                                 ⋱  
   -0.040423+0.0im    0.0661698+0.032648im  

しかし、実際のところ上記の数値計算で得られる固有ベクトル $X$ は近似の値 $\bX\in \C^{n\times n}$ (以下、近似固有ベクトルという) である。しかし、近似固有ベクトルを並べた行列を使って、対角化したものについては、対角優位性が期待できる。この行列を $G$ とすると、

$$
G := \bX^{-1}A\bX
$$

となり、行列 $G$ の対角成分は行列 $A$ の固有値の近似となる。

また、対角優位性とは、ある行列$A=(a_{ij})$について、

$$
\left|a_{i i}\right|>\sum_{{j} \colon j \neq i}\left|a_{i j}\right| \quad(i=1,2, \ldots, n)
$$

であること。つまり、行列のどの対角成分の絶対値も、それと同じ列にある非対角成分の和よりも大きい時。

In [4]:
G = inv(X)*A*X; #G
norm(diag(G) - λ,Inf) # Gの対角成分は固有値λにほぼ一致する

6.034386413298531e-14

#### (2)$A\cdot\bX$の包含$C\in\IC^{n\times n}$を求める。

``方法1``各演算を区間演算に変更した方式(参考：[interval_dot-mul.ipynb](https://www.risk.tsukuba.ac.jp/~takitoshi/tutorial/interval_dot-mul.html "タイトル"))

In [5]:
using IntervalArithmetic

iA = map(Interval, A);
iX = map(Interval, X);

@btime C1 = $iA * $iX;

LoadError: LoadError: UndefVarError: @btime not defined
in expression starting at In[5]:6

In [41]:
real(C1)

100×100 Array{Interval{Float64},2}:
 [-2.22817, -2.22816]      …  [-1.12834, -1.12833]
  [2.05301, 2.05302]           [0.770477, 0.770478]
 [-0.928074, -0.928073]        [1.51408, 1.51409]
 [-0.815416, -0.815415]        [0.17174, 0.171741]
  [0.376184, 0.376185]        [-0.114519, -0.114518]
 [-0.749198, -0.749197]    …   [0.651971, 0.651972]
  [2.81687, 2.81688]           [1.20245, 1.20246]
 [-0.0734694, -0.0734693]     [-0.0438072, -0.0438071]
 [-0.34671, -0.346709]        [-0.300528, -0.300527]
  [0.0294944, 0.0294945]       [1.11102, 1.11103]
 [-1.74325, -1.74324]      …   [1.03995, 1.03996]
  [0.213074, 0.213075]         [0.195834, 0.195835]
 [-0.150519, -0.150518]       [-1.13716, -1.13715]
   ⋮                       ⋱  
  [0.0900892, 0.0900893]       [0.690261, 0.690262]
  [0.457129, 0.45713]         [-1.52083, -1.52082]
 [-1.73796, -1.73795]      …  [-0.4791, -0.479099]
  [0.294049, 0.29405]         [-0.617965, -0.617964]
 [-1.55344, -1.55343]          [0.139657, 0.139658]
  [0

In [37]:
imag(C1)

100×100 Array{Interval{Float64},2}:
 [0, 0]   [0.0356235, 0.0356236]     …  [0, 0]  [0, 0]
 [0, 0]  [-0.406769, -0.406768]         [0, 0]  [0, 0]
 [0, 0]  [-0.176554, -0.176553]         [0, 0]  [0, 0]
 [0, 0]   [0.346179, 0.34618]           [0, 0]  [0, 0]
 [0, 0]   [0.599175, 0.599176]          [0, 0]  [0, 0]
 [0, 0]  [-0.263535, -0.263534]      …  [0, 0]  [0, 0]
 [0, 0]   [0.104969, 0.10497]           [0, 0]  [0, 0]
 [0, 0]   [0.0942661, 0.0942662]        [0, 0]  [0, 0]
 [0, 0]   [0.197508, 0.197509]          [0, 0]  [0, 0]
 [0, 0]   [0.55774, 0.557741]           [0, 0]  [0, 0]
 [0, 0]  [-0.204599, -0.204598]      …  [0, 0]  [0, 0]
 [0, 0]  [-0.165138, -0.165137]         [0, 0]  [0, 0]
 [0, 0]  [-0.173883, -0.173882]         [0, 0]  [0, 0]
      ⋮                              ⋱          
 [0, 0]   [0.147363, 0.147364]          [0, 0]  [0, 0]
 [0, 0]   [0.29806, 0.298061]           [0, 0]  [0, 0]
 [0, 0]   [0.0200258, 0.0200259]     …  [0, 0]  [0, 0]
 [0, 0]  [-0.00914094, -0.00914093]

In [61]:
@show max(maximum(radius.(real(C1[:]))),maximum(radius.(imag(C1[:]))))

max(maximum(radius.(real(C1[:]))), maximum(radius.(imag(C1[:])))) = 3.7192471324942744e-15


3.7192471324942744e-15

``方法2``BLASを使う。int_num_linalg.jlの呼び出し。この方が早い。

In [15]:
using BenchmarkTools
include("int_num_linalg.jl"); # mm_ufpを使用する
# function mm_real_interval(A,B) # A, B: real matrices
#     C_mid, C_rad = imm_ufp(mid.(A),radius.(A),mid.(B),radius.(B));
#     return C_mid .± C_rad
# end
# function mm_comp_interval_alt(A,B) # A, B: complex matrices
# #     (Ar + Ai)*(Br + Bi) = (Ar*Br - Ai*Bi) + im()
#     Ar = real(A); Ai = imag(A);
#     Br = real(B); Bi = imag(B);
#     Cr = mm_real_interval(Ar, Br) - mm_real_interval(Ai, Bi);
#     Ci = mm_real_interval(Ar, Bi) + mm_real_interval(Ai, Br);
#     return Cr + im*Ci;
# end
function mm_comp_interval(A,B) # A, B: complex matrices (not interval)
    C_mid, C_rad = mm_ufp(A,B);
    return (real(C_mid) .± C_rad) + im*(imag(C_mid) .± C_rad)
end

# @time C1 = mm_comp_interval_alt(iA,iX);
@time C = mm_comp_interval(A,X); # A, X: complex matrices (not interval)

  0.201898 seconds (332.43 k allocations: 14.978 MiB)


In [17]:
max(maximum(radius.(real(C[:]))),maximum(radius.(imag(C[:]))))

9.103828801926284e-14

#### 3)区間連立1次方程式 $\hat XG=C$ の解集合の包含 $G\supset\hat X^{-1}\cdot C$ を求める。

In [49]:
function verifylss_ib(A,ib) # verify the solution element-wisely
    b = mid.(real(ib)) + mid.(imag(ib))*im;
    x̄ = A\b;
    n = length(x̄);
    R = inv(A);
    #########
    #C_mid, C_rad = mm_ufp(R,A);
    G = Matrix{Float64}(I, n, n) - mm_comp_interval(R,A);
    α = opnorm(G,Inf)# Interval arithmetic
    #########
    if α < 1
        x̄ = map(Interval,x̄);
        r = A*x̄ - ib; # Interval arithmetic
        Rr = R*r;
        err = abs.(Rr) + supremum(norm(Rr,Inf))/(1-α)*(abs.(G)*ones(n)); # Interval arithmetic
    else
        println("Oh my way, verification is failed...")
        err = nan;
    end
    return x = (real(x̄) .± supremum.(err)) + im*(imag(x̄) .± supremum.(err))
end

verifylss_ib (generic function with 1 method)

In [53]:
G = similar(C);
for i = 1:n
    G[:,i] = verifylss_ib(X,C[:,i]);
end
real(G)

100×100 Array{Interval{Float64},2}:
 [-10.627, -10.6269]           …  [-1.13763e-12, 1.08021e-12]
  [-3.55853e-12, 3.56038e-12]     [-3.31708e-12, 3.30127e-12]
  [-3.55853e-12, 3.56038e-12]     [-3.34071e-12, 3.30339e-12]
  [-2.49773e-12, 2.50091e-12]     [-2.32006e-12, 2.34947e-12]
  [-3.30444e-12, 3.30623e-12]     [-3.09508e-12, 3.10892e-12]
  [-3.30444e-12, 3.30623e-12]  …  [-3.09404e-12, 3.1362e-12]
  [-2.90712e-12, 2.91473e-12]     [-2.78067e-12, 2.75303e-12]
  [-2.90712e-12, 2.91473e-12]     [-2.764e-12, 2.75568e-12]
  [-3.52756e-12, 3.52188e-12]     [-3.25582e-12, 3.31768e-12]
  [-3.52756e-12, 3.52188e-12]     [-3.26588e-12, 3.32628e-12]
  [-4.84495e-12, 4.84742e-12]  …  [-4.63311e-12, 4.53473e-12]
  [-4.84495e-12, 4.84742e-12]     [-4.62198e-12, 4.5284e-12]
  [-5.33161e-12, 5.33087e-12]     [-5.02182e-12, 5.01385e-12]
    ⋮                          ⋱  
  [-3.18687e-12, 3.19528e-12]     [-2.98856e-12, 3.01634e-12]
  [-4.3154e-12, 4.32716e-12]      [-4.00724e-12, 4.04364e-12]
  [

In [85]:
R=inv(X)
R[1,1]

0.05942443289846145 + 6.869504964868156e-16im

In [87]:
C=real(C)+imag(C)
C[1,1]

[-2.22817, -2.22816]

#### 4)$G\in\IC^{n\times n}$に対してゲルシュゴリンの定理を適用することを考える。


## ゲルシュゴリンの包含定理
行列 $A=(A_{ij})\in \C^{n\times n}$について、$A$の全ての固有値$\lambda_i$は、$\bigcup_{1\le i \le n}  U_i$の内部に存在する。

$$
A = \bigcup_{1\le i \le n}  U_i
$$

ただし、

$$
U_i = \{z\in\mathbb{C}:|z-a_{ii}|\le \sum_{j\neq i}|a_{ij}|\}
$$

である。

円盤領域$U_i$をゲルシュゴリン円板といい、Aが強い優対角性を持つとき、Aの対角成分がAの固有値の良い近似となる。

今回の場合、$G=(g_{ij})$の対角成分$[g_{ii}]$を$c_i$、半径を$\rm{rad(g_{ii})}$とすると、

$$
\begin{array}{ll}
\mbox{ゲルシュゴリン円の中心:}&c_i\\
\mbox{ゲルシュゴリン円の半径:}&r_i=\sum_{j\neq i}\rm{mag}(g_{ij})+\rm{rad}(g_{ii})\\
\end{array}
$$

となる。

対角優位
http://www.math.ritsumei.ac.jp/~yasutomi/jugyo/Numerical_Analysis/note5.pdf

計算途中がでる
http://www.cas.cmc.osaka-u.ac.jp/~paoon/misc/julia/post/cg/